# Ingredients Ranked By Average Pairing Rank (r = 2)

In [ ]:
from pymongo import MongoClient
import sys

client = MongoClient()
db = client['yummly']
combo_coll = db['combinationsTop50k']

ranked = []

cursor = combo_coll.find({"r": 1}, no_cursor_timeout=True)
for record in cursor:
    ingredient = record['_id']
    pipeline = [
        {
            "$match": {
                "r": 2,
                "ingredients": {
                    "$in": [ingredient]
                }
            }
        },
        {
            "$group": {
                "_id": None,
                "avg_score": {
                    "$avg": "$score"
                }
            }
        }
    ]
    avg = combo_coll.aggregate(pipeline).next()['avg_score']
    ranked.append((ingredient, avg))
    ranked = sorted(ranked, key = lambda ingredient: ingredient[1], reverse=True)
    sys.stdout.write("\r{message}".format(message=ranked[:3]))
    sys.stdout.flush()
    
cursor.close()

ranked = sorted(ranked, key = lambda ingredient: ingredient[1])
for item in ranked:
    print(item)

[('yuzu juice', 0.136365411491179), ('celery microgreens', 0.13636541149117898), ('gluten-free pretzel', 0.06044481854747594)]